In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

from fredapi import Fred

import graph_templates


In [2]:
# Get the API key and set the path to it
API_KEY_PATH = "fred_api_key.txt" 
fred = Fred(api_key_file = API_KEY_PATH)

# Get the team template and set it as the default
pio.templates.default = 'fed_2025'

# Get the path to the graph_output folder (one folder back from the notebook)
GRAPH_OUTPUT_PATH = "../graph_output/"

In [3]:
fred.get_series_info(series_id="CPIAUCSL")

test_srs = fred.get_series(
    series_id="CORESTICKM159SFRBATL",
    observation_start= dt.date(1980, 1, 1),
    observation_end= dt.date(2025, 1, 1)
)

test_srs_2 = fred.get_series(
    series_id="CPILFESL",
    observation_start= dt.date(1980, 1, 1),
    observation_end= dt.date(2025, 1, 1)
).pct_change(periods=12) * 100

test_srs_3 = fred.get_series(
    series_id="CPIAUCSL",
    observation_start= dt.date(1980, 1, 1),
    observation_end= dt.date(2025, 1, 1)
).pct_change(periods=12) * 100


joined_df = pd.concat([test_srs, test_srs_2, test_srs_3], axis=1, join='inner')
joined_df.columns = ['Sticky Core CPI', 'Core CPI', 'Headline CPI']

In [8]:
test_srs.index

DatetimeIndex(['1980-01-01', '1980-02-01', '1980-03-01', '1980-04-01',
               '1980-05-01', '1980-06-01', '1980-07-01', '1980-08-01',
               '1980-09-01', '1980-10-01',
               ...
               '2024-04-01', '2024-05-01', '2024-06-01', '2024-07-01',
               '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01',
               '2024-12-01', '2025-01-01'],
              dtype='datetime64[ns]', length=541, freq=None)

In [4]:
fig = go.Figure()

for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

fig.update_layout(
    title = dict(text = 'Test Series<br><sup>CPIAUCSL from FRED</sup>'),
    xaxis_title="Date",
    yaxis_title="Value"
)

fig.update_xaxes(
    tickformat='%Y-%m',
    ticklabelmode='instant',
    tickangle=45
)

fig.write_image(
    file = GRAPH_OUTPUT_PATH + "/test_title.png",
    format = 'png',
    width = 1200,
    height = 500
)
